In [1]:
!pip install --user --upgrade numpy==1.13.3
!pip install --user surprise

Requirement already up-to-date: numpy==1.13.3 in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s2b9-36b6fb815be3bf-8a4d16ccce63/.local/lib/python3.5/site-packages


In [6]:
import pandas as pd
import numpy as np
from surprise import Reader, Dataset, evaluate, KNNBasic

import warnings; warnings.simplefilter('ignore')

In [3]:
from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_30ffad4bf0ec4ca2b39a54de347c401f(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_19f9e546xxxxxxxxxxxxxxxxxx','domain': {'id': '325df8e3b4d44421a6f9a1d915b530f3'},
            'password': 'xxxxxxx/xxxxxxxx'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)

ratings = pd.read_csv(get_object_storage_file_with_credentials_30ffad4bf0ec4ca2b39a54de347c401f('factprediction', 'fact_ratings.csv'))
ratings.head()

,userId,factId,rating
0,100,50,5.0
1,100,51,5.0
2,100,52,5.0
3,100,53,5.0
4,100,54,1.0


In [4]:
reader = Reader()
data = Dataset.load_from_df(ratings[['userId', 'factId', 'rating']], reader)
data.split(n_folds=3)

In [8]:
model = KNNBasic()

training_data = data.build_full_trainset()
model.train(training_data)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [10]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact

service_path = 'https://ibm-watson-ml.mybluemix.net'
username = '556ef63b-xxx-xxxx-xxxx-xxxxxxxx'
password = 'a1d617c3-xxxx-xxxx-xxxx-xxxxxxxx'

ml_repository_client = MLRepositoryClient(service_path)
ml_repository_client.authorize(username, password)

model_artifact = MLRepositoryArtifact(model, training_data=training_data, name="Fact Prediction Model")
model_saved = ml_repository_client.models.save(model_artifact)

ValueError: Invalid type for ml_artifact: KNNBasic